# mise en place du code final

- Gestion de récupération de la liste des artistes et groupes sans doublons

In [1]:
# Kernel 3.11.9 pour mon VSC
# on importe les librairies
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time # on importe le time pour faire des pauses vis à vis de l'api qui nous donne un rate limit

# Premier compte : client_id='5a63812d960f4789a86499ecb6073348', client_secret='31cf73ebc10247fcb28175c1a777f34a'
# deuxième compte : client_id='f692473a0b79476398c28bc4c2248aec', client_secret='dcffa96d20e54c9c85016ffc5bc1fcdb'

# on créé une connexion avec les données du compte.
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='5a63812d960f4789a86499ecb6073348',

                                               client_secret='31cf73ebc10247fcb28175c1a777f34a',

                                               redirect_uri='http://localhost:8080',

                                               scope='user-library-read'))

In [2]:
# récupération des données via l'api

# nous avons besoin de deux listes pour récupérer les noms et les id spotify
df_list_artist = pd.DataFrame(columns=['artist_name','artist_id'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_artist_year( year):    
    
    # nous avons besoin de deux listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    artist_name = []
    id_artist_spotify = []
    
    # on est limité à 1000 retours max que l'on fait à travers des pages de 50 retours via un for
    for i in range(0,1000,50):
        
        # on lance l'API pour récupérer les données de la page en cours avec l'année en paramètre (en str)
        track_results = sp.search(q='year:'+str(year), type='album', limit=50,offset=i)
        
        # on intègre les données récupérées dans les bonnes listes
        # on parcours les différents albums de la page
        for i, t in enumerate(track_results['albums']['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/search
            artist_name.append(t['artists'][0]['name'])
            id_artist_spotify.append(t['artists'][0]['id'])
    
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_artist = pd.DataFrame({'artist_name' : artist_name, 'artist_id' : id_artist_spotify})
    df_artist.drop_duplicates(inplace=True)
    df_list_artist2 = pd.concat([df_list_artist, df_artist])
    
    # on affiche un petit message pour chaque année récupérée
    print(f'Année {year} effectuée ! avec {df_artist.shape[0]} lignes sur l"année et {df_list_artist2.shape[0]} lignes sur la liste')
    return df_list_artist2

In [85]:
for year in range(2000, 2026, 1):
    df_list_artist = search_artist_year(year)
    df_list_artist.drop_duplicates(inplace=True)

Année 2000 effectuée ! avec 864 lignes sur l"année et 864 lignes sur la liste
Année 2001 effectuée ! avec 834 lignes sur l"année et 1698 lignes sur la liste
Année 2002 effectuée ! avec 847 lignes sur l"année et 2418 lignes sur la liste
Année 2003 effectuée ! avec 871 lignes sur l"année et 3049 lignes sur la liste
Année 2004 effectuée ! avec 841 lignes sur l"année et 3558 lignes sur la liste
Année 2005 effectuée ! avec 846 lignes sur l"année et 4041 lignes sur la liste
Année 2006 effectuée ! avec 815 lignes sur l"année et 4473 lignes sur la liste
Année 2007 effectuée ! avec 850 lignes sur l"année et 4934 lignes sur la liste
Année 2008 effectuée ! avec 828 lignes sur l"année et 5344 lignes sur la liste
Année 2009 effectuée ! avec 833 lignes sur l"année et 5781 lignes sur la liste
Année 2010 effectuée ! avec 795 lignes sur l"année et 6172 lignes sur la liste
Année 2011 effectuée ! avec 823 lignes sur l"année et 6593 lignes sur la liste
Année 2012 effectuée ! avec 826 lignes sur l"année et

In [97]:
df_list_artist[df_list_artist['artist_name'] == 'ZZ Top']

,artist_name,artist_id
604,ZZ Top,2AM4ilv6UzW0uMRuqKtDgN


In [ ]:
df_list_artist.to_csv('liste_artistes.csv', index=False)

- récupération de la liste des albums

In [14]:
# pour tester, on récupère le df créé juste au dessus en utilisant la première cellule
data_artistes = pd.read_csv('info_artist_0-1754.csv')

# vu que c'est le nouveau fichier, je ne veux que les id avec les noms
data_artistes = data_artistes[['artist_name','artist_id']]

In [15]:
data_artistes

,artist_name,artist_id
0,The Naked And The Dead,16HZrqib5dArFNZ7YgukCU
1,Latin Quarter,2Wd6dWP1QK0ofkTvTWjhTZ
2,The English Concert,2Oz3z7PPQUMU2KNYgcO1sA
3,Raven,1EH9eSje47IiRyVsq3gfkl
4,El Pelos,0olRBwuh8P1adSECxfoQpH
...,...,...
1749,Hassan Shamaizadeh,12TmVMYwkTjWDEPNhnDE4G
1750,Dismember,09OwM7iXyoFAOzQUlWeDgB
1751,Levellers,1TrwMxRrrlk0hZxJkw4jUF
1752,Sarah Cunningham,78qL3Rn9v8FjkBn7rdmjQW


In [16]:
# récupération des données via l'api pour la partie Albums

# nous avons besoin de listes pour récupérer les infos des albums
df_list_album = pd.DataFrame(columns=['artist_id','album_name','album_id','release_date','total_tracks','url_image'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_album_artist(artist_id):    
    
    # nous avons besoin de deux listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    album_name = []
    album_id = []
    release_date = []
    total_tracks = []
    url_image = []

    
    # on reste sur le même shéma que la fonction précédente même si on ne risque pas de se retrouver avec trop de page. 
    # je préfère anticiper le groupe qui aura plus de 50 albums et donc deux pages. 
    for i in range(0,1000,50):
        
        # on lance l'API pour récupérer les données de la page en cours avec l'année en paramètre (en str)
        albums_results = sp.artist_albums(artist_id, include_groups='album', limit=50,offset=i)
        
        # on intègre les données récupérées dans les bonnes listes
        # on parcours les différents albums de la page
        for i, t in enumerate(albums_results['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/get-an-artists-albums

            album_name.append(t['name'])
            album_id.append(t['id'])
            release_date.append(t['release_date'])
            total_tracks.append(t['total_tracks'])
            try:
                url_image.append(t['images'][0]['url'])
            except:
                url_image.append("")

    
    # # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_album = pd.DataFrame({'artist_id':artist_id,\
                             'album_name' : album_name,\
                             'album_id' : album_id,\
                             'release_date' : release_date,\
                             'total_tracks' : total_tracks,\
                             'url_image' : url_image})
    df_list_album2 = pd.concat([df_list_album, df_album])
    time.sleep(1)
    # on retourne le df créé
    return df_list_album2

In [17]:
# le test fonctionne bien pour un artiste
test = search_album_artist('1Z8ODXyhEBi3WynYw0Rya6')
test

,artist_id,album_name,album_id,release_date,total_tracks,url_image
0,1Z8ODXyhEBi3WynYw0Rya6,3 Feet High and Rising (35th Anniversary),2VKV04dOyLIEkBz9iaccP7,2024-03-01,29,https://i.scdn.co/image/ab67616d0000b273d4e7eb...
1,1Z8ODXyhEBi3WynYw0Rya6,Buhloone Mindstate (30th Anniversary),07K9cv8WdI6KWQsb9QtZR5,2023-09-21,18,https://i.scdn.co/image/ab67616d0000b2732ccd1e...
2,1Z8ODXyhEBi3WynYw0Rya6,FIRST SERVE (Instrumental version),6spi3faBh90zAqXiz9f7FZ,2020-11-07,12,https://i.scdn.co/image/ab67616d0000b2737d5de6...
3,1Z8ODXyhEBi3WynYw0Rya6,FIRST SERVE (2020 remastered version),1DJ5jL2YvtMlR18u0CzMNn,2020-11-07,16,https://i.scdn.co/image/ab67616d0000b2737ff4b6...
4,1Z8ODXyhEBi3WynYw0Rya6,and the Anonymous Nobody... (Instrumental),2Wr5LY8SHnirKifPmhmDu2,2016-08-26,16,https://i.scdn.co/image/ab67616d0000b273423f4c...
5,1Z8ODXyhEBi3WynYw0Rya6,and the Anonymous Nobody...,73USNxSgFncLAsOAWfIA4J,2016-08-26,17,https://i.scdn.co/image/ab67616d0000b2736e977e...
6,1Z8ODXyhEBi3WynYw0Rya6,The Grind Date,6pA5vsn22nBSarlpib38QR,2004-10-05,13,https://i.scdn.co/image/ab67616d0000b2730f487a...
7,1Z8ODXyhEBi3WynYw0Rya6,"Live At Tramps, NYC, 1996",4VKGbQiwRdvkZJOHrISqvU,2004-05-15,15,https://i.scdn.co/image/ab67616d0000b27300a5d3...
8,1Z8ODXyhEBi3WynYw0Rya6,AOI: Bionix,6GeCGp8XF2mRxxuZaiAaEU,2001-12-04,18,https://i.scdn.co/image/ab67616d0000b273bc73ac...
9,1Z8ODXyhEBi3WynYw0Rya6,Art Official Intelligence: Mosaic Thump,4SdFt1Q5HSQIk4lxPgjaPM,2000-08-08,18,https://i.scdn.co/image/ab67616d0000b273423b3b...


In [98]:
# on récupère la liste des albums des 5 premiers artistes de notre DF
for i in range(5):
    df_list_album = search_album_artist(data.loc[i,'artist_id'])

In [ ]:
# ici on récupère la totalité des lignes de notre fichier d'artiste avec un fichier de 12000 lignes c'est 3h. J'ai ajouté un print pour connaitre le nom de l'artiste ajouté
for i in range (len(data)):
    df_list_album = search_album_artist(data.loc[i,'artist_id'])
    print(f"Albums de {data.loc[i,'artist_name']} récupérés")

Albums de The Naked And The Dead récupérés
Albums de Latin Quarter récupérés
Albums de The English Concert récupérés
Albums de Raven récupérés


In [ ]:
# export du df pour les test même si il n'est pas finalisé. 
df_list_album.to_csv('list_artist_album.csv', index=False)

- récupération des informations des artistes

In [2]:
# on ouvre le fichier de artistes
data = pd.read_csv('df_list_artist_p2.csv')

In [3]:
data

,artist_name,artist_id
0,The Naked And The Dead,16HZrqib5dArFNZ7YgukCU
1,Latin Quarter,2Wd6dWP1QK0ofkTvTWjhTZ
2,The English Concert,2Oz3z7PPQUMU2KNYgcO1sA
3,Raven,1EH9eSje47IiRyVsq3gfkl
4,El Pelos,0olRBwuh8P1adSECxfoQpH
...,...,...
13749,JC Reyes,0FwnPHExlRRxEZPLAi5tmG
13750,Sheltoh,0BeBk0t2FZqz0X4aUHkBs2
13751,Travy,3Jlb9WCVsroYGv8TKOGLly
13752,Godwin,2mzRsCFnmBdMKFgSCVqE3Y


In [4]:
# récupération des données via l'api pour la partie Albums
# via cette API https://developer.spotify.com/documentation/web-api/reference/get-an-artist
# création d'une fonction récupérant le dataframe des artistes avec leurs id pour en récupérer les informations
# on utilise un dataframe en entrée qui doit contenir au moins deux colonnes. une avec les id artiste ('artist_id')
# et l'autre avec les nom d'artiste (artist_name) . on pourrait faire sans le second vu que le nom est dans l' API
# a voir si on le fait qu'avec une liste d'ID ?
def search_infos_artist(df):
    # nous avons besoin de listes pour récupérer les infos des artistes
    artist_name = []
    artist_id = []
    artist_popularity = []
    artist_followers = []
    artist_genres = []
    artist_image = []
    # on parcours la liste des id des artistes et grace au df on peut récupérer le nom.
    for ele, val in enumerate(df['artist_id']):
        # pour chaque artistes on lance une requête sur l'API pour récupérer les données
        artist_results = sp.artist(val)
        # on créé une liste des informations de l'liste_artistes
        artist_name.append(artist_results['name'])
        artist_id.append(val)
        artist_popularity.append(artist_results['popularity'])
        artist_followers.append(artist_results['followers']['total'])
        artist_genres.append(artist_results['genres'])
        try:
            artist_image.append(artist_results['images'][0]['url'])
        except:
            artist_image.append("")
        time.sleep(5)
        print(artist_results['name'], " ajouté")
    
    # une fois les lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_artist = pd.DataFrame({'artist_name':artist_name,\
                                'artist_id' : artist_id,\
                                'popularity' : artist_popularity,\
                                'artist_followers' : artist_followers,\
                                'genres' : artist_genres,\
                                'image' : artist_image})
    # on patiente 1 seconde entre deux DF pour limiter l'accès à l'API

    # on retourne le df créé
    return df_artist

In [5]:
# test sur 1000 lignes de notre data. Le but sera de le faire une une seule fois pour les 11000 artiste. Soit 3h. 
df = search_infos_artist(data.head(1754))

The Naked And The Dead  ajouté
Latin Quarter  ajouté
The English Concert  ajouté
Raven  ajouté
El Pelos  ajouté
Šerif Konjević  ajouté
Alisha  ajouté
Orchestre Fayçal  ajouté
Tommy Shaw  ajouté
Kix  ajouté
Zedek Mouloud  ajouté
Golden Bough  ajouté
EAV (Erste Allgemeine Verunsicherung)  ajouté
Nicolas de Angelis  ajouté
Danny Williams  ajouté
Altzükütarrak  ajouté
The Ray Miranda Orchestra  ajouté
Tilahun Gessesse  ajouté
Óscar Medina  ajouté
Parabellum  ajouté
Michael Brook  ajouté
Benjamin Lew  ajouté
Frank Boeijen Groep  ajouté
Sir Colin Davis  ajouté
Deul Guk Hwa  ajouté
Martinelli  ajouté
Raji  ajouté
Ami Ozaki  ajouté
André Cymone  ajouté
Майк Науменко  ajouté
The Kane Gang  ajouté
The Firm  ajouté
Frederick the Great  ajouté
Lone Justice  ajouté
Tobias Hume  ajouté
Owen Paul  ajouté
BULLDOZER  ajouté
Mango  ajouté
Marco Armani  ajouté
Destruction  ajouté
Café Moulu  ajouté
Justin Hayward  ajouté
Orlando Vera Cruz  ajouté
Yvonne Chaka Chaka  ajouté
Sabine Paturel  ajouté
Schoolly

In [8]:
df.to_csv('info_artist_0-1754.csv', index=False)

In [11]:
df.sort_values(by='artist_followers', ascending=True).head(25)

,artist_name,artist_id,popularity,artist_followers,genres,image
16,The Ray Miranda Orchestra,4FBIpMIor7ksX2miuLRaEH,20,0,[],https://i.scdn.co/image/ab67616d0000b2736d8486...
922,Enrico Caruso,1yqvi08TXELaUOWf9nsy2A,11,4,[opera],https://i.scdn.co/image/ab67616d0000b273d9db71...
385,Rosie Vela,5jPqffCEpPL6WW9RguQUMP,7,14,[],https://i.scdn.co/image/ab67616d0000b273cf0228...
891,Didier Rimaud,6Myzsj5O3bbq6zmYusDA2f,2,28,[],https://i.scdn.co/image/ab67616d0000b273cda7ee...
1259,Paulo Roberto,6HlXvZsx8ZvbwYQTsA8rHa,8,29,[],https://i.scdn.co/image/ab6761610000e5ebdab7c5...
319,Manoj - Gyan,57eLXpSw3fVa6g8i8L9pOO,0,32,[],https://i.scdn.co/image/ab67616d0000b273bd419e...
274,Les Echos du Pays d\'Auge,0W2y2oYnp7HKmB1yJlED5I,2,33,[],https://i.scdn.co/image/ab67616d0000b273869a25...
40,Café Moulu,1hWo8Pq4XfdD5NmU4t7tKJ,2,35,[],https://i.scdn.co/image/ab67616d0000b2733e9976...
599,Rachid Ferrache,4pUgmIxR2my2zDHzurWhLz,5,41,[],https://i.scdn.co/image/ab67616d0000b273017617...
258,Roland Kovac Rhythm & Strings,5Uf62D8i84n1XpQZSmub9z,18,44,[],https://i.scdn.co/image/ab67616d0000b27305bf0f...


In [105]:
data.head()

,artist_name,artist_id
0,Linkin Park,6XyY86QOPPrYVGvF9ch6wz
1,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb
2,Deftones,6Ghvu1VvMGScGpOUJBAHNH
3,Alice Deejay,2tbvDi9eXf9XXp06LupkED
4,Radiohead,4Z8W4fKeB5YxbusRsdQVPb


In [102]:
df

,artist_name,artist_id,popularity,artist_followers,genres,image
0,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,88,28641293,"[nu metal, rap metal, rock, alternative metal]",https://i.scdn.co/image/ab6761610000e5ebc7e6bd...
1,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb,75,6850757,"[nu metal, rap metal, alternative metal, rap r...",https://i.scdn.co/image/4654955890cb62a6abe9da...


In [127]:
data

,artist_name,artist_id
0,Linkin Park,6XyY86QOPPrYVGvF9ch6wz
1,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb
2,Deftones,6Ghvu1VvMGScGpOUJBAHNH
3,Alice Deejay,2tbvDi9eXf9XXp06LupkED
4,Radiohead,4Z8W4fKeB5YxbusRsdQVPb
...,...,...
11305,Karnalh,0YwSU2NF2cXIME0vXeJVmL
11306,Francesco Yates,5X1JzPIIonP3u9lA580pPT
11307,XXKATUSJINSUX,0Zkkh7YLEO75PmH7vwgO7X
11308,Love and Deepspace,1s7vHZwbZZSJdp4tXlYOtC


- récupération de la liste des titres par albums (version par liste d'albums pour éviter le time-out, cf partie suivante)

In [21]:
# on conserve le df créé précédemment. ou on le récupère

df_list_album = pd.read_csv('list_artist_album.csv') 

# création de liste de 10 éléments pour l'API des titres. On va faire une liste de liste

liste_albums_global = list(df_list_album['album_id'])

# définition d'une fonction itérative utilisant le yield (Contrairement à return, yield ne termine pas la fonction ; 
# il suspend son exécution et reprend là où elle s’était arrêtée.)  cf : https://blog.alphorm.com/maitriser-yield-python

def creation_nb_liste(lst,nb):  # création d'une fonction qui prends deux arguments, une liste et un nombre. On utilisera ce nombre pour déterminer le nombre d'élément de la sous liste
    for ele in range(0,len(lst),nb): # on parcours la liste avec un pas du nombre nb
        yield lst[ele:ele + nb]  # a chaque itération on slicer la liste de nb valeurs pour créer une sous liste.

In [22]:
# on génére une liste du résultat de la fonction précédente
liste_album_par_dix = list(creation_nb_liste(liste_albums_global, 20))

In [23]:
# test de la fonction de création d'une liste de liste de 10 albums pour éviter le time-out
liste_album_par_dix

[['4R6FV9NSzhPihHR0h4pI93',
  '1x6p39FFIESOq8q8B4oQXN',
  '494aUGKXMH5ruXtR3O1a3H',
  '1X472EvsSqH09RyrqbtyXU',
  '3Q9wXhEAX7NYCPP0hxIuDz',
  '28DUZ0itKISf2sr6hlseMy',
  '0zcNtUZ2oEpRmiDuWBFvcX',
  '5Eevxp2BCbWq25ZdiXRwYd',
  '6hU9JCoqq4GjYq86dQ1o9b',
  '3XB2yloP7l00tEUmaODtVi',
  '2FUsvD1bw53HGOjAg56vRD',
  '7qRKpkP0jSOlBMLYZhAMCh',
  '4XHIjbhjRmqWlosjj5rqSI',
  '0hJ3A7sih1AXDoMjXHLr7Q',
  '7IYqppCBhR5z9z8JqgXuxi',
  '4flcwtqnLoKZJ2wrCp1aJq',
  '1FiRqhpAowNK8gTl5sOhxZ',
  '4fy0SXW5G8evBCo2A3kn02',
  '5uvXx5ZQswNRFCdHR521YZ',
  '72seWTJF9U5SljizfyF2ZK'],
 ['4JtIVJRA342O0YoAchen5Q',
  '7pgs38iLfEqUtwgCRgvbND',
  '0ZBE7rVC0zKFVt5osvXlnz',
  '0f7R0jf0pcTb6K6IVVPcMD',
  '2NbO8RRVTVEUjHaxUVdMDT',
  '6hPkbAV3ZXpGZBGUvL6jVM',
  '2lZ9RqGnGXH8gtH3DK02tJ',
  '2xYsmEzewmcKVPlYXOXVeh',
  '73SsqFGUL7Xns1ZK3weKiK',
  '1W990lM14o82dzCUGEtvTd',
  '6tbD7TXKlegmp644gE2iKe',
  '3hkyt2ofB7tGrsbxwBaCE9',
  '3MXrPupQnGD3XQLRbxDyW9',
  '3HCCUaRSjHSFOe4fqE0BiP',
  '5c6tiN4r4GwOTkuJKLzW2X',
  '3I34EprFC0rlAO4p

In [69]:
for ele in liste_album_par_dix:
    print(ele)

['4R6FV9NSzhPihHR0h4pI93', '1x6p39FFIESOq8q8B4oQXN', '494aUGKXMH5ruXtR3O1a3H', '1X472EvsSqH09RyrqbtyXU', '3Q9wXhEAX7NYCPP0hxIuDz', '28DUZ0itKISf2sr6hlseMy', '0zcNtUZ2oEpRmiDuWBFvcX', '5Eevxp2BCbWq25ZdiXRwYd', '6hU9JCoqq4GjYq86dQ1o9b', '3XB2yloP7l00tEUmaODtVi', '2FUsvD1bw53HGOjAg56vRD', '7qRKpkP0jSOlBMLYZhAMCh', '4XHIjbhjRmqWlosjj5rqSI', '0hJ3A7sih1AXDoMjXHLr7Q', '7IYqppCBhR5z9z8JqgXuxi', '4flcwtqnLoKZJ2wrCp1aJq', '1FiRqhpAowNK8gTl5sOhxZ', '4fy0SXW5G8evBCo2A3kn02', '5uvXx5ZQswNRFCdHR521YZ', '72seWTJF9U5SljizfyF2ZK']
['4JtIVJRA342O0YoAchen5Q', '7pgs38iLfEqUtwgCRgvbND', '0ZBE7rVC0zKFVt5osvXlnz', '0f7R0jf0pcTb6K6IVVPcMD', '2NbO8RRVTVEUjHaxUVdMDT', '6hPkbAV3ZXpGZBGUvL6jVM', '2lZ9RqGnGXH8gtH3DK02tJ', '2xYsmEzewmcKVPlYXOXVeh', '73SsqFGUL7Xns1ZK3weKiK', '1W990lM14o82dzCUGEtvTd', '6tbD7TXKlegmp644gE2iKe', '3hkyt2ofB7tGrsbxwBaCE9', '3MXrPupQnGD3XQLRbxDyW9', '3HCCUaRSjHSFOe4fqE0BiP', '5c6tiN4r4GwOTkuJKLzW2X', '3I34EprFC0rlAO4pKNeX1t', '57KFwKnC0ZjaIQGnhE7ops', '0VEFy5MsBiq0u2lWL0OwOd', '3tsXyEbUQ

In [25]:
# on récupère une liste simple de 20 valeur
liste_album_par_dix[0:1][0]

['4R6FV9NSzhPihHR0h4pI93',
 '1x6p39FFIESOq8q8B4oQXN',
 '494aUGKXMH5ruXtR3O1a3H',
 '1X472EvsSqH09RyrqbtyXU',
 '3Q9wXhEAX7NYCPP0hxIuDz',
 '28DUZ0itKISf2sr6hlseMy',
 '0zcNtUZ2oEpRmiDuWBFvcX',
 '5Eevxp2BCbWq25ZdiXRwYd',
 '6hU9JCoqq4GjYq86dQ1o9b',
 '3XB2yloP7l00tEUmaODtVi',
 '2FUsvD1bw53HGOjAg56vRD',
 '7qRKpkP0jSOlBMLYZhAMCh',
 '4XHIjbhjRmqWlosjj5rqSI',
 '0hJ3A7sih1AXDoMjXHLr7Q',
 '7IYqppCBhR5z9z8JqgXuxi',
 '4flcwtqnLoKZJ2wrCp1aJq',
 '1FiRqhpAowNK8gTl5sOhxZ',
 '4fy0SXW5G8evBCo2A3kn02',
 '5uvXx5ZQswNRFCdHR521YZ',
 '72seWTJF9U5SljizfyF2ZK']

In [90]:
# récupération des données via l'api pour la partie titres

# nous avons besoin de listes pour récupérer les infos des albums
df_track_album = pd.DataFrame(columns=['album_name','album_id','album_type','total_tracks','release_date','release_date_precision','upc','label','popularity','images','disc_number','track_number','track_id','track_name','duration_ms'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_track_album(liste_albums):    
    
    # nous avons besoin de listes pour récupérer les informations
    
    album_name = []
    album_id = []
    album_type = []
    total_tracks = []
    release_date = []
    release_date_precision = []
    upc = []
    label = []
    popularity = []
    images = []
    disc_number = []
    track_number = []
    track_id = []
    track_name = []
    duration_ms = []
        
    # on lance l'API pour récupérer les données, pas de notion de pages ici. Nous avons des sous listes pour avoir tous les titres et nous fonctionnons avec 
    # une liste des albums recherchés.
    albums_track_results = sp.albums(liste_albums)

    # On parcours le nombre d'album que nous aurons en retour
    for ligne,valeur in enumerate(albums_track_results['albums']):
    # on intègre les données récupérées dans les bonnes listes, on utilise un enumérate pour lister les différents titres des albums. 
    # il y aura deux partie, une avec des append sur les infos globales toujours identiques et une partie sur les infos de chaque titre

        for i, t in enumerate(albums_track_results['albums'][ligne]['tracks']['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/get-multiple-albums

            # on récupère les quelques données propres à l'album sans utiliser la boucle mais on le fait dans la boucle pour avoir des tableaux de même valeurs pour le df:
            album_name.append(albums_track_results['albums'][ligne]['name'])
            album_id.append(albums_track_results['albums'][ligne]['id'])
            album_type.append(albums_track_results['albums'][ligne]['album_type'])
            total_tracks.append(albums_track_results['albums'][ligne]['total_tracks'])
            release_date.append(albums_track_results['albums'][ligne]['release_date'])
            release_date_precision.append(albums_track_results['albums'][ligne]['release_date_precision'])
            upc.append(albums_track_results['albums'][ligne]['external_ids']['upc'])
            label.append(albums_track_results['albums'][ligne]['label'])
            popularity.append(albums_track_results['albums'][ligne]['popularity'])
            
            try:
                images.append(albums_track_results['albums'][ligne]['images'][0]['url'])
            except:
                images.append('')    
            # on récupère les données propre au titre en utilisant la boucle et ses valeurs
            disc_number.append(t['disc_number'])
            track_number.append(t['track_number'])
            track_id.append(t['id'])
            track_name.append(t['name'])
            duration_ms.append(t['duration_ms'])

        
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_track = pd.DataFrame({'album_name':album_name,\
                            'album_id' : album_id,\
                            'album_type' : album_type,\
                            'total_tracks' : total_tracks,\
                            'release_date' : release_date,\
                            'release_date_precision' : release_date_precision,\
                            'upc' : upc,\
                            'label' : label,\
                            'popularity' : popularity,\
                            'images' : images,\
                            'disc_number' : disc_number,\
                            'track_number' : track_number,\
                            'track_id' : track_id,\
                            'track_name' : track_name,\
                            'duration_ms' : duration_ms})
        
        
    df_list_track_album2 = pd.concat([df_track_album, df_track])
    
    # on retourne le df créé
    return df_list_track_album2

In [91]:
test = search_track_album(['494aUGKXMH5ruXtR3O1a3H'])

In [92]:
test

,album_name,album_id,album_type,total_tracks,release_date,release_date_precision,upc,label,popularity,images,disc_number,track_number,track_id,track_name,duration_ms
0,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,1,4XaV4XMROzY1G242gB3Q4K,Crawling (Instrumental),209034
1,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,2,46ppIcCWeyCeNS6BbveSc0,Faint (Instrumental),162726
2,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,3,4ySeTJOYb31uzSdIMw0U3x,Numb / Encore (Instrumental),205233
3,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,4,4B1Uaz5w9HGXtIigc3zyQa,Papercut (Instrumental),184346
4,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,5,3zruk8xsxvYcLOsYjFNl8E,Breaking the Habit (Instrumental),195388
5,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,6,6RUv6gg1Jv1NMT8N5Jv2Wd,In the End (Instrumental),216346
6,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,7,6rmBv18D2JgQy0GWIRwwST,Bleed It Out (Instrumental),164466
7,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,8,7sg2aQZFGgIhDuOZVUGWYO,Somewhere I Belong (Instrumental),214017
8,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,9,6y7mwRBftYgFiAk2QRYOi1,Waiting for the End (Instrumental),231157
9,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,album,20,2024-06-28,day,093624840831,Warner Records,48,https://i.scdn.co/image/ab67616d0000b273cfa827...,1,10,0L0yA1cbvbeXUYkrCGa1Zd,CASTLE OF GLASS (Instrumental),205394


In [66]:
id = ['566osTxDsfrtdBxPDMGufx','0f7R0jf0pcTb6K6IVVPcMD','3I34EprFC0rlAO4pKNeX1t'] # pour test
# test sur 20 albums
df = pd.DataFrame()
for ele in liste_album_par_dix:
    df = pd.concat([df,search_track_album(ele)])

In [70]:
df.isrc.value_counts()

isrc
{'upc': '093624856085'}      50
{'upc': '093624893219'}      50
{'upc': '191404116654'}      34
{'upc': '634904086855'}      23
{'upc': '093624947035'}      22
                             ..
{'upc': '634904078768'}       8
{'upc': '634904078461'}       8
{'upc': '093624949732'}       8
{'upc': '00602577027680'}     7
{'upc': '093624949725'}       7
Name: count, Length: 64, dtype: int64

In [16]:
df.tail(20)

,album_name,album_id,album_type,total_tracks,release_date,label,popularity,images,disc_number,track_number,track_id,track_name,duration_ms
27,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,5,76RAlQcfuQknnQFruYDj6Q,Bones,189000
28,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,6,1tZcw7GtIqviL32bzaKdSo,(Nice Dream),233226
29,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,7,1dyTcli07c77mtQK3ahUZR,Just,234800
30,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,8,0jyikFM0Umv0KlnrOEKtTG,My Iron Lung,276426
31,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,9,5XuU9htN358NTMCcqRvfDV,Bullet Proof ... I Wish I Was,208733
32,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,10,6UO72VSXEONxdfLyABihs9,Black Star,247040
33,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,11,1elQc2QcyuBkI8FUIbNvcy,Sulk,222960
34,The Bends,35UJLpClj5EDrhpNIi4DFg,album,12,1995-03-13,XL Recordings,73,https://i.scdn.co/image/ab67616d0000b2739293c7...,1,12,2QwObYJWyJTiozvs0RI7CF,Street Spirit (Fade Out),253853
35,Pablo Honey,3gBVdu4a1MMJVMy6vwPEb8,album,12,1993-02-22,XL Recordings,75,https://i.scdn.co/image/ab67616d0000b273df55e3...,1,1,5KZ0qobWEFl892YjIC02SE,You,208666
36,Pablo Honey,3gBVdu4a1MMJVMy6vwPEb8,album,12,1993-02-22,XL Recordings,75,https://i.scdn.co/image/ab67616d0000b273df55e3...,1,2,70LcF31zb1H0PyJoS1Sx1r,Creep,238640


In [ ]:
df.loc[df.album_name == 'Hybrid Theory (20th Anniversary Edition)']

- version par id albums qui est trop gourmandes en ressources. Je me suis fait jeter de spotify avec un time-out de 8h

In [4]:
# on conserve le df créé précédemment. ou on le récupère

df_list_album = pd.read_csv('list_artist_album.csv') 

In [8]:
df_list_album

,artist_id,album_name,album_id,release_date,total_tracks,url_image
0,6XyY86QOPPrYVGvF9ch6wz,From Zero,4R6FV9NSzhPihHR0h4pI93,2024-11-15,11,https://i.scdn.co/image/ab67616d0000b273b11a54...
1,6XyY86QOPPrYVGvF9ch6wz,From Zero: A Cappellas,1x6p39FFIESOq8q8B4oQXN,2024-11-14,11,https://i.scdn.co/image/ab67616d0000b273fcef4c...
2,6XyY86QOPPrYVGvF9ch6wz,Papercuts: Instrumentals,494aUGKXMH5ruXtR3O1a3H,2024-06-28,20,https://i.scdn.co/image/ab67616d0000b273cfa827...
3,6XyY86QOPPrYVGvF9ch6wz,Papercuts,1X472EvsSqH09RyrqbtyXU,2024-04-12,20,https://i.scdn.co/image/ab67616d0000b273163d1c...
4,6XyY86QOPPrYVGvF9ch6wz,Meteora 20th Anniversary Edition,3Q9wXhEAX7NYCPP0hxIuDz,2023-04-07,89,https://i.scdn.co/image/ab67616d0000b273ca2e09...
...,...,...,...,...,...,...
59,4Z8W4fKeB5YxbusRsdQVPb,Amnesiac,1HrMmB5useeZ0F5lHrMvl0,2001-03-12,11,https://i.scdn.co/image/ab67616d0000b273863e0e...
60,4Z8W4fKeB5YxbusRsdQVPb,Kid A,6GjwtEZcfenmOf6l18N7T7,2000-10-02,11,https://i.scdn.co/image/ab67616d0000b2736c7112...
61,4Z8W4fKeB5YxbusRsdQVPb,OK Computer,6dVIqQ8qmQ5GBnJ9shOYGE,1997-05-28,12,https://i.scdn.co/image/ab67616d0000b273c8b444...
62,4Z8W4fKeB5YxbusRsdQVPb,The Bends,35UJLpClj5EDrhpNIi4DFg,1995-03-13,12,https://i.scdn.co/image/ab67616d0000b2739293c7...


In [116]:
df_list_album.loc[4,'album_id'] # attention nous avons plusieurs lignes à 0 !!!!  ça doit être la même sur le df original il faudra reset les index quelque part

'3Q9wXhEAX7NYCPP0hxIuDz'

In [9]:
# récupération des données via l'api pour la partie titres

# nous avons besoin de listes pour récupérer les infos des albums
df_list_track_album = pd.DataFrame(columns=['artist_name','artist_id','album_id','track_id','disc_number','track_number','track_name','duration_ms'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_track_album(album_id):    
    
    # nous avons besoin de deux listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    artist_name = []
    artist_id = []
    track_id = []
    disc_number = []
    track_number = []
    track_name = []
    duration_ms = []


    
    # on reste sur le même shéma que la fonction précédente même si on ne risque pas de se retrouver avec trop de page. 
    # je préfère anticiper le groupe qui aura plus de 50 albums et donc deux pages. 
    for i in range(0,1000,50):
        
        # on lance l'API pour récupérer les données de la page en cours avec l'année en paramètre (en str)
        albums_track_results = sp.album_tracks(album_id, limit=50,offset=i)
        
        # on intègre les données récupérées dans les bonnes listes
        # on parcours les différents albums de la page
        for i, t in enumerate(albums_track_results['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks

            artist_name.append(t['artists'][0]['name'])
            artist_id.append(t['artists'][0]['id'])
            track_id.append(t['id'])
            disc_number.append(t['disc_number'])
            track_number.append(t['track_number'])
            track_name.append(t['name'])
            duration_ms.append(t['duration_ms'])# diviser par /60000 pour l'avoir en minutes
            
    
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_track_album = pd.DataFrame({'artist_name':artist_name,\
                             'artist_id' : artist_id,\
                             'album_id' : album_id,\
                             'track_id' : track_id,\
                             'disc_number' : disc_number,\
                             'track_number' : track_number,\
                             'track_name' : track_name,\
                             'duration_ms' : duration_ms})
    df_list_track_album2 = pd.concat([df_list_track_album, df_track_album])
    
    # on retourne le df créé
    return df_list_track_album2

In [ ]:
# le test fonctionne bien pour un album
test = search_track_album('1X472EvsSqH09RyrqbtyXU')
test

In [ ]:
# on récupère la liste des titres des 5 premiers albums de notre DF initial
for i in range(64):
    df_list_track_album = search_track_album(df_list_album.loc[i,'album_id'])
    # on ajoute une pause vu que l'API ne souhaite pas que l'on puisse faire trop de requêtes à la suite
    print(f"Albun n°{i} nommé {df_list_album.loc[i,'album_name']} vient d'être ajouté")
    time.sleep(1)  # 1 sec de pause entre deux requêtes
    

In [13]:
df_list_track_album

,artist_name,artist_id,album_id,track_id,disc_number,track_number,track_name,duration_ms
0,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,4R6FV9NSzhPihHR0h4pI93,1uhces29ApKuQWFdpiUsvH,1,1,From Zero (Intro),22174
1,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,4R6FV9NSzhPihHR0h4pI93,1EDPVGbyPKJPeGqATwXZvN,1,2,The Emptiness Machine,190427
2,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,4R6FV9NSzhPihHR0h4pI93,5mFZipkX1HZ4Idz5LOWpzq,1,3,Cut the Bridge,228606
3,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,4R6FV9NSzhPihHR0h4pI93,2HBBM75Xv3o2Mqdyh1NcM0,1,4,Heavy Is the Crown,167012
4,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,4R6FV9NSzhPihHR0h4pI93,3k4gjmYNJugQRlh2aounZ9,1,5,Over Each Other,170445
...,...,...,...,...,...,...,...,...
9,Alice Deejay,2tbvDi9eXf9XXp06LupkED,7wyEph8JrTyNFNPmRCu3pU,1s1RZvjFx09wMDVmOfobAK,1,10,No More Lies,215806
10,Alice Deejay,2tbvDi9eXf9XXp06LupkED,7wyEph8JrTyNFNPmRCu3pU,6zmF1ahOh37GJyITvqYTL8,1,11,I Can See (See It In Your Eyes),224490
11,Alice Deejay,2tbvDi9eXf9XXp06LupkED,7wyEph8JrTyNFNPmRCu3pU,0tjlB7o1wjE7D7OJNz552I,1,12,Everything Begins With An E,259204
12,Alice Deejay,2tbvDi9eXf9XXp06LupkED,7wyEph8JrTyNFNPmRCu3pU,1ytIizeTXX75N4qY8331yv,1,13,Got To Get Away,220659
